# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096934


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:43,  3.70s/it]

Rendering models:  10%|█         | 3/30 [00:08<01:16,  2.83s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:54,  2.10s/it]

Rendering models:  17%|█▋        | 5/30 [00:09<00:44,  1.77s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:19<00:01,  2.41it/s]

Rendering models:  93%|█████████▎| 28/30 [00:20<00:00,  2.01it/s]

Rendering models:  97%|█████████▋| 29/30 [00:20<00:00,  1.88it/s]

Rendering models: 100%|██████████| 30/30 [00:21<00:00,  1.82it/s]

not-logged-in-3b0b929f4475fd276561    0.002073
Jnursssmith5263                       0.020078
not-logged-in-673e227f920b24558cc1    0.001360
ellivc                                0.001498
taysish58                             0.001190
24declanfm                            0.001219
awright5                              0.002948
framosuw                              0.001179
Mickfurry                             1.499806
not-logged-in-5f30b2bb9b7f9d49da41    0.465219
AdamMorse                             0.003871
not-logged-in-314d48142a3a122ba666    0.002040
DarioB                                0.001386
clairedeu                             0.001323
withadnotat                           0.001296
acapirala                             0.001234
not-logged-in-3ae935e3afaee843f530    0.159291
Lavadude                              0.104864
not-logged-in-4937d52fc3b062ae0d43    0.002385
w7250369                              0.001734
MatheusSilva                          0.002184
equidad1     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())